# Práctica final de Normativa y Regulación

El desarrollo consiste en utilizar dos dataset o conjunto, para hacer uso de datos abiertos(Open Data) de datos del catalogo de datos [Open Data Canarias](http://www.opendatacanarias.es/) y [TENERIFEDATA](http://www.tenerifedata.es) en concreto el conjunto de datos [alojamientos turisticos de tenerife](http://www.opendatacanarias.es/datos/dataset/3f637e7d-d875-48c8-9337-6f16f767c655/resource/a6864f3e-acda-40fd-92a1-ae292e713399/download/alojamientos.csv) y el conjunto de datos [paradas de guaguas de tenerife](http://www.tenerifedata.es/dataset/transporte/resource/8e0664aa-e67d-47e0-adec-54b37fd21e38)
*****

**Caracteristicas de estos conjuntos**
* **Paradas de guaguas de tenerife:**
    * **Resumen**: Recursos georeferenciados relacionados con la movilidad: aparcamientos, paradas de guaguas,...
    * **Última actualización:** desconocido
    * **Fecha de creación:** 2 Años
    * **Formatos:** JSON,CSV
    * **Licencia:** [Creative Commons CCZero](http://opendefinition.org/licenses/cc-zero/)
    * **Autor:** desconocido
    * **Mantenedor:** desconocido
    * **Versión:** desconocido
    * ** Cobertura Geográfica:** Tenerife
    * **Idioma:** español
* **Alojamientos turistico:**
    * **Resumen:** Contiene toda la información acerca de los alojamientos registrados por Turismo de Tenerife. La información que aporta este dataset es recogida desde el punto de vista de interés para el turista por lo que puede haber algún que no esté correcto. Los recursos se encuentran en los formatos: XML, JSON, CSV.<br>
    **NOTA:** Los ficheros en RDF se han realizado en colaboración con el grupo TARO de investigación de la Universidad de La Laguna.
    * **Última actualización:** Julio 3, 2015
    * ** Fecha de creación:** Junio 18,2014
    * **Formatos:** CSV, JSON, XML, RDF
    * **Licencia:** [Open Data Commons Attribution License](http://opendefinition.org/licenses/odc-by/)
    * **Autor:** SPET, Turismo de Tenerife S.A
    * **Mantenedor:** SPET, Turismo de Tenerife S.A
    * **Versión:** 1.1
    * ** Cobertura Geográfica:** Tenerife
    * **Idioma:** español  
    

## Aplicación
* ** Autor:** Orlandy Ariel Sánchez A.
* ** Asignatura:** Normativa y Regulación.
* ** Lenguaje: ** Python
* **Licencia:** [Creative Commons Reconocimiento 4.0 Internacional](https://creativecommons.org/licenses/by/4.0/)

Para el desarrollo de la aplicación se hará uso de datos abiertos(Open Data) utilizando la herramienta [Jupyter Notebook](http://jupyter.org/).
Se hará uso de las siguientes librerias:
* Para el texto se utilizará Markdown
* Para el código Python
* Se utilizarán las librerias:
    * [IPython](https://ipython.org/)
    * [gpxpy](https://pypi.python.org/pypi/gpxpy/0.7.0)
    * [Pandas](http://pandas.pydata.org/)
    * [Folium](https://folium.readthedocs.io/en/latest/)
    
**NOTA:** TENER EN CUENTA DE QUE LAS LIBRERÍAS TIENEN QUE ESTAR INSTALADAS EN LA EN LA MÁQUINA PARA PODER EJECUTARLAS

### Paradas de guaguas de Tenerife


In [7]:
from IPython.display import HTML
import gpxpy.geo
import pandas as pd
import folium

In [6]:
#Carga de datos paradas
datosParadas = pd.read_csv('./paradas.csv')
mapaParadas = folium.Map(location=[28.45, -16.2333], tiles='Stamen Terrain', zoom_start=10)

#Carga de datos alojamietnos.
datosAlojamientos = pd.read_csv('alojamientos.csv')
mapaAlojamientos = folium.Map(location=[28.45, -16.2333], tiles='Stamen Terrain', zoom_start=10)

In [9]:
#Verificación que los datos sean de tipo float
def isfloat(value):
      try:
        float(value)
        return True
      except ValueError:
        return False

** Método para calcular la distancia entre dos puntos, haciendo uso de la librería gpxpy.geo que hace uso de la [Fórmula del haversine](https://es.wikipedia.org/wiki/F%C3%B3rmula_del_haversine)**

In [64]:
arrayCoordenadas1 = []
arrayCoordenadas2 = []
def distancia(lat1, lng1, lat2, lng2):
    dist = gpxpy.geo.haversine_distance(lat1, lng1, lat2, lng2)
    if(dist < 20): # menor de 50 Km
        #print([lat1,lng1])
        #rint("coor")
        #rint([lat2,lng2])
        arrayCoordenadas1.append([lat1,lng1])
        arrayCoordenadas2.append([lat2,lng2])

In [65]:
#extracción de los datos de las coordenadas, filtrados a la zona metropolitana
arrayAloj = []
for eq in datosAlojamientos.iterrows():
    if(eq[1]['ows_Zona'] == 'Metropolitana'):
        coordenadas = eq[1]['ows_Georeferencia']
        if(not isfloat(coordenadas)):
            if(not coordenadas == ""):
                line = coordenadas.strip("(").strip(")").strip(" ")
                line = line.replace(" ", "").split(",")
                arrayAloj.append(line)
arrayAlojamientos = []
for coordenadas in arrayAloj:
    x = float(coordenadas[0])
    y = float(coordenadas[1])
    arrayAlojamientos.append([x,y])

In [66]:
arrayParadas = []
for eq in datosParadas.iterrows():
    if(isfloat(eq[1][1])):
        x= float(eq[1][1])
    if(isfloat(eq[1][0])):
        y=float(eq[1][0])
    #print ([x,y])
    arrayParadas.append([x,y])

In [67]:
for i in range(len(arrayParadas)):
    for j in range(len(arrayAlojamientos)):
        distancia(arrayParadas[i][0],arrayParadas[i][1],arrayAlojamientos[j][0],arrayAlojamientos[j][1])

In [82]:
mapaDistancias = folium.Map(location=[28.45, -16.2333], tiles='Stamen Terrain', zoom_start=11)
for i in range(len(arrayCoordenadas1)):
    marker1 = folium.Marker(location=[arrayCoordenadas1[i][0],arrayCoordenadas1[i][1]],
                            icon=folium.Icon(color='red'))
    mapaDistancias.add_children(marker1)
for i in range(len(arrayCoordenadas1)):
    marker2 = folium.Marker(location=[arrayCoordenadas2[i][0],arrayCoordenadas2[i][1]],
                            icon=folium.Icon(color='blue'))
    mapaDistancias.add_children(marker2)
mapaDistancias.save('distancia.html')


In [139]:
HTML('<iframe src=distancia.html width=100% height=350></iframe>')

## Referencia visual de donde están las paradas y los alojamientos.

In [ ]:
icon_url = 'iconoAlojamiento.png'
icon_size=(40, 40)
for j in range(len(arrayAlojamientos)):
    print([arrayAlojamientos[i][0],arrayAlojamientos[i][1]])
    marca = folium.Marker(location=[arrayAlojamientos[i][0],arrayAlojamientos[i][1]],
                              icon = folium.features.CustomIcon(icon_url,icon_size))
    mapaAlojamientos.add_children(marca)

mapaAlojamientos.save('hoteles.html')

In [ ]:
HTML('<iframe src=hoteles.html width=100% height=350></iframe>')

In [153]:
icon_url = 'iconoTaxi.png'
icon_size=(40, 40)
for j in range(len(arrayParadas)):
    marca = folium.Marker(location=[arrayParadas[i][0],arrayParadas[i][1]],
                              icon = folium.features.CustomIcon(icon_url,icon_size))
    mapaParadas.add_children(marca)

mapaParadas.save('paradas.html')

In [ ]:
HTML('<iframe src=paradas.html width=100% height=350></iframe>')